1. Label data using Named Entity Recognition
2. Split data into train and test
3. train model
4. test model

In [27]:
import pandas as pd

import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span, DocBin

from scholarly import scholarly

import requests
import mimetypes
import os

import uuid

import PyPDF2

from PyPDF2 import PdfReader
import concurrent.futures

from spacy import displacy
import random
#from tqdm import tqdm
from tqdm.notebook import tqdm
import sys
import warnings
from contextlib import redirect_stderr
from io import StringIO

In [2]:
ROOT_FOLDER = '../../../'

In [3]:
DATA_FOLDER = '../../../data/'

In [4]:
DETECT_ML_MODEL_FILES_FOLDER = '../../../detect_ml_model_files/'

In [5]:
data_df = pd.read_parquet(f'{DATA_FOLDER}detect_ml_models.parquet')

drop records where the file_name is Null

In [6]:
data_df = data_df.dropna()

drop records where language is not English

In [7]:
data_df = data_df.query('language == "en"')

In [8]:
data_df

,title,abstract,year,url,author_id,query,file_name,file_path,language
3,Ridge regression: applications to nonorthogona...,This paper is an exposition of the use of ridg...,1970,https://scholar.archive.org/work/zedzhecp75hdj...,"[, ]",Ridge Regression,520946c9-4f85-4fd1-bfe5-2305c315cb1b.pdf,../../../detect_ml_model_files/520946c9-4f85-4...,en
4,Lecture notes on ridge regression,well-known results on ridge regression. The cu...,2015,https://arxiv.org/pdf/1509.09169,[qglWXEQAAAAJ],Ridge Regression,7f813b9f-7b00-4610-b3fe-30b85ad26b51.pdf,../../../detect_ml_model_files/7f813b9f-7b00-4...,en
5,A critique of some ridge regression methods,These techniques are consequently all subject ...,1980,http://economics-files.pomona.edu/garysmith/pa...,"[8CzBuBYAAAAJ, ]",Ridge Regression,5ed3ef6d-6030-47a6-9ddf-363a2f09571c.pdf,../../../detect_ml_model_files/5ed3ef6d-6030-4...,en
6,Ridge regression learning algorithm in dual va...,of the Ridge Regression procedure. It allows u...,1998,https://eprints.soton.ac.uk/258942/1/Dualrr_IC...,"[OEd0rvkAAAAJ, uoWoR4gAAAAJ, GJE29ekAAAAJ]",Ridge Regression,0acd09a1-4322-48eb-907d-b48498c2d89c.pdf,../../../detect_ml_model_files/0acd09a1-4322-4...,en
7,Ridge regression: Biased estimation for nonort...,"For this reason, estimation and analysis built...",1970,https://mineracaodedados.wordpress.com/wp-cont...,"[, ]",Ridge Regression,578f8b95-0643-4dcf-8565-00c5d655caa2.pdf,../../../detect_ml_model_files/578f8b95-0643-4...,en
...,...,...,...,...,...,...,...,...,...
1294,Llama: Open and efficient foundation language ...,"We introduce LLaMA, a collection of foundation...",2023,https://arxiv.org/pdf/2302.13971,"[xImarzoAAAAJ, 9nPunCEAAAAJ, aL3MllMAAAAJ, GLf...",LLaMA,535ca623-e366-4ebd-ab52-8e7e585033a0.pdf,../../../detect_ml_model_files/535ca623-e366-4...,en
1308,LLAMA language aptitude tests: The manual,The Llama tests are the result of this work. T...,2005,https://citeseerx.ist.psu.edu/document?repid=r...,[],LLaMA,927466f3-11db-4836-bcb2-ea4b4c4695f5.pdf,../../../detect_ml_model_files/927466f3-11db-4...,en
1309,The llama 3 herd of models,empirical evaluation of Llama 3. We find that ...,2024,https://arxiv.org/pdf/2407.21783,"[KJNUEgkAAAAJ, w56EascAAAAJ, , wjJnkcAAAAAJ]",LLaMA,3b96923f-fd1a-485c-a4a2-cce732c26886.pdf,../../../detect_ml_model_files/3b96923f-fd1a-4...,en
1318,Code llama: Open foundation models for code,from Llama 2 to Code Llama in Llama models cl...,2023,https://arxiv.org/pdf/2308.12950,"[6vht2iwAAAAJ, jOwTwm4AAAAJ, , UAx_woYAAAAJ, T...",LLaMA,094c84c1-c9d6-4302-a100-311f513862da.pdf,../../../detect_ml_model_files/094c84c1-c9d6-4...,en


The query column represents the machine learning model name that was used to search Google Scholar.

There should be five papers per model.

Add a new column to represent an identifier/order for the paper within each query

In [9]:
#data_df['counter'] = data_df.groupby('query').cumcount() + 1
data_df.loc[:, 'counter'] = data_df.groupby('query').cumcount() + 1

In [10]:
data_df

,title,abstract,year,url,author_id,query,file_name,file_path,language,counter
3,Ridge regression: applications to nonorthogona...,This paper is an exposition of the use of ridg...,1970,https://scholar.archive.org/work/zedzhecp75hdj...,"[, ]",Ridge Regression,520946c9-4f85-4fd1-bfe5-2305c315cb1b.pdf,../../../detect_ml_model_files/520946c9-4f85-4...,en,1
4,Lecture notes on ridge regression,well-known results on ridge regression. The cu...,2015,https://arxiv.org/pdf/1509.09169,[qglWXEQAAAAJ],Ridge Regression,7f813b9f-7b00-4610-b3fe-30b85ad26b51.pdf,../../../detect_ml_model_files/7f813b9f-7b00-4...,en,2
5,A critique of some ridge regression methods,These techniques are consequently all subject ...,1980,http://economics-files.pomona.edu/garysmith/pa...,"[8CzBuBYAAAAJ, ]",Ridge Regression,5ed3ef6d-6030-47a6-9ddf-363a2f09571c.pdf,../../../detect_ml_model_files/5ed3ef6d-6030-4...,en,3
6,Ridge regression learning algorithm in dual va...,of the Ridge Regression procedure. It allows u...,1998,https://eprints.soton.ac.uk/258942/1/Dualrr_IC...,"[OEd0rvkAAAAJ, uoWoR4gAAAAJ, GJE29ekAAAAJ]",Ridge Regression,0acd09a1-4322-48eb-907d-b48498c2d89c.pdf,../../../detect_ml_model_files/0acd09a1-4322-4...,en,4
7,Ridge regression: Biased estimation for nonort...,"For this reason, estimation and analysis built...",1970,https://mineracaodedados.wordpress.com/wp-cont...,"[, ]",Ridge Regression,578f8b95-0643-4dcf-8565-00c5d655caa2.pdf,../../../detect_ml_model_files/578f8b95-0643-4...,en,5
...,...,...,...,...,...,...,...,...,...,...
1294,Llama: Open and efficient foundation language ...,"We introduce LLaMA, a collection of foundation...",2023,https://arxiv.org/pdf/2302.13971,"[xImarzoAAAAJ, 9nPunCEAAAAJ, aL3MllMAAAAJ, GLf...",LLaMA,535ca623-e366-4ebd-ab52-8e7e585033a0.pdf,../../../detect_ml_model_files/535ca623-e366-4...,en,1
1308,LLAMA language aptitude tests: The manual,The Llama tests are the result of this work. T...,2005,https://citeseerx.ist.psu.edu/document?repid=r...,[],LLaMA,927466f3-11db-4836-bcb2-ea4b4c4695f5.pdf,../../../detect_ml_model_files/927466f3-11db-4...,en,2
1309,The llama 3 herd of models,empirical evaluation of Llama 3. We find that ...,2024,https://arxiv.org/pdf/2407.21783,"[KJNUEgkAAAAJ, w56EascAAAAJ, , wjJnkcAAAAAJ]",LLaMA,3b96923f-fd1a-485c-a4a2-cce732c26886.pdf,../../../detect_ml_model_files/3b96923f-fd1a-4...,en,3
1318,Code llama: Open foundation models for code,from Llama 2 to Code Llama in Llama models cl...,2023,https://arxiv.org/pdf/2308.12950,"[6vht2iwAAAAJ, jOwTwm4AAAAJ, , UAx_woYAAAAJ, T...",LLaMA,094c84c1-c9d6-4302-a100-311f513862da.pdf,../../../detect_ml_model_files/094c84c1-c9d6-4...,en,4


In [11]:
# nlp = spacy.blank("en")
# ruler = nlp.add_pipe("entity_ruler")

#nlp = spacy.load("en_core_web_sm")
#ruler = nlp.add_pipe("entity_ruler", before="ner")

nlp = spacy.load("en_core_web_sm", exclude=["ner"])  # Exclude the default NER component
ruler = nlp.add_pipe("entity_ruler", last=True)

# Load the EntityRuler patterns from a file
ruler.from_disk(f"{DATA_FOLDER}ml_entity_ruler_patterns")

Lets take the first four files for a model for training and the last one for test. This will give us a 80/20 split of files at least.

The query results from Google Scholar are in descending order of popularity. This should mean that the files are not necessarily in order of published date, which is good.

Further analysis is required to look at the terms in each file and create a more balanced training/test split.



In [12]:
training_files = data_df.query('counter < 5')['file_path'].tolist()

In [13]:
len(training_files)

396

In [14]:
test_files = data_df.query('counter == 5')['file_path'].tolist()

In [15]:
len(test_files)

99

There should be 4 times more training_files than test_files, lets test that

In [16]:
len(test_files) * 4 == len(training_files)

True

In [17]:
data_df.groupby(['query']).count().reset_index().query('counter != 5')['query'].values

array([], dtype=object)

Find terms in file to use as labels

In [28]:
def process_pdf_file(pdf_file_path):
    docs = []
    try:
        with open(pdf_file_path, "rb") as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            for page_num, page in enumerate(pdf_reader.pages):
                # Extract text from the page
                text = page.extract_text()
                doc = nlp(text)
    
                # Extract sentences with ML_METHOD entities
                filtered_sentences = [
                    sentence.text
                    for sentence in doc.sents
                    if any(ent.label_ == "ML_METHOD" for ent in sentence.ents)
                ]
    
                if len(filtered_sentences) > 0:
                    # Create a new text containing only those sentences
                    filtered_text = " ".join(filtered_sentences)
    
                    # Create a new Doc object with the filtered text
                    filtered_doc = nlp(filtered_text)
                    docs.append(filtered_doc)
    except Exception as e:
        print(f"Error checking file {pdf_file_path}: {e}")
    
    return docs

def create_labelled_docs(files):
    all_docs = []

        # Suppress or redirect stderr to capture warning messages
    stderr_buffer = StringIO()
    with redirect_stderr(stderr_buffer):
        with concurrent.futures.ProcessPoolExecutor() as executor:  # Use ProcessPoolExecutor for CPU-bound tasks
            #results = list(executor.map(process_pdf_file, files))
            results = list(tqdm(executor.map(process_pdf_file, files), total=len(files), desc="Processing files"))

    # Flatten the list of results
    for doc_list in results:
        all_docs.extend(doc_list)

    # Optional: Display warnings if needed
    warnings_output = stderr_buffer.getvalue()
    if warnings_output:
        print("Warnings captured during processing:", file=sys.stderr)
        print(warnings_output, file=sys.stderr)

    return all_docs

# Generate training data

In [19]:
training_docs = create_labelled_docs(files=training_files)

In [20]:
if len(training_docs) < 1:
    print('There are no labels created for the requested files!!!')
else:
    # save it in a format spacy can use
    training_doc_bin = DocBin(docs=training_docs)
    #training_doc_bin.to_disk("./train.spacy")
    training_doc_bin.to_disk("./train_detect_ml_models.spacy")    

Lets visualise a random training doc

In [21]:
# Set a seed for repeatability
seed = 2048
random.seed(seed)

random_integer = random.randint(0, len(training_docs))

# Render the entities in a Jupyter Notebook or as HTML
options = {"colors": {"ORG": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "ML_METHOD": "#f6c5be"}}
displacy.render(training_docs[random_integer], style="ent", options=options, jupyter=True)


# Generate test data

In [29]:
test_docs = create_labelled_docs(files=test_files)

Processing files:   0%|          | 0/99 [00:00<?, ?it/s]

In [30]:
if len(test_docs) < 1:
    print('There are no labels created for the requested files!!!')
else:
    # save it in a format spacy can use
    test_doc_bin = DocBin(docs=test_docs)
    test_doc_bin.to_disk("./test_detect_ml_models.spacy")    

In [32]:
# Set a seed for repeatability
seed = 33
random.seed(seed)

random_integer = random.randint(0, len(test_docs))

# Render the entities in a Jupyter Notebook or as HTML
options = {"colors": {"ORG": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "ML_METHOD": "#f6c5be"}}
displacy.render(test_docs[random_integer], style="ent", options=options, jupyter=True)